In [3]:
#Импортируем необходимые модули  из Pullenti и объявляем переменные
from pullenti_wrapper.langs import (
    set_langs,
    RU
)
set_langs([RU])
from pullenti_wrapper.processor import (
    Processor,
    GEO,
    ADDRESS
)
from pullenti_wrapper.referent import Referent
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv("data_test.csv" , sep=',' , encoding='cp1251').head(100) # top 100                 , index_col=0 - индекс 
df['mark_6']=df['mark_6'].apply(str)
df['mark_6_do'] = np.nan
df.head() 

In [ ]:
processor = Processor([GEO, ADDRESS])
#Текст, содержащий адреса
#text = 'Новокузнецк автотранспортная 45. Новокузнецк Рокоссовского 16. Новокузнецк дачное товарищество Робинзон. Новокузнецк Ильинское шоссе'#'При решении возникшей проблемы я уже было отступила, но оператор связи своими смссообщениями всё таки подвинула меня для окончательного решения возникшей ситуации /Москва, улица Марьинский парк'#Москва, улица Марьинский парк ,д.21,кор.2'#Железногорск, Энтузиастов 10Б"#'Москва, улица Марьинский парк ,д.21,кор.2'
#Функция формирования словаря с адресами
def display_shortcuts(referent, level=0):
    tmp = {}
    a = ""
    b = ""
    for key in referent.__shortcuts__:
        value = getattr(referent, key)
        if value in (None, 0, -1):
            continue
        if isinstance(value, Referent):
            display_shortcuts(value, level + 1)
        else:
            if key == 'type':
                a = value 
            if key == 'name':
                b = value
            if key == 'house':
                a = "дом"
                b = value
                tmp[a] = b
            if key == 'flat':
                a = "квартира"
                b = value
                tmp[a] = b
            if key == 'corpus':
                a = "корпус"
                b = value
                tmp[a] = b
    tmp[a] = b
    addr.append(tmp)


x = -1
for i, row in df.iterrows(): #перебор
    x = x + 1
    if row['mark_6'].isdigit() : #проверка на число
        continue #
    else:
        addr = []       
        #print(x) #ошибка
        #print(row['mark_6'])        
        #print(result)
        try: #обращение к  функции 
            result = processor(row['mark_6'])
            referent = result.matches[0].referent
            display_shortcuts(referent)
            #print(addr)
            ##addr2 = ', '.join([f'{key.capitalize()}:{value}'for key , value in addr[0].items()])
            #print('!1',addr2)#','.join(addr))
            st=[]
            i=0
            while i<len(addr):
                for key,value in addr[i].items():
                    st+=key, value
                i+=1
            df.loc[x,'mark_6_do'] = ' '.join(st) #запись 
            #print(' '.join(st))#','.join(addr))
        except:
            continue
df  


In [6]:
df.to_excel('Pullenti_res.xlsx')